In [1]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: fineGrained).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [2]:
!pip install diffusers --upgrade
!pip install invisible_watermark transformers accelerate safetensors

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 25.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 21.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 82.4 MB/s eta 0:00:00


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
# TODO: Reorder imports
import sys
import torch
from diffusers import DiffusionPipeline, StableDiffusionXLPipeline
from diffusers.models.attention import Attention
from diffusers.models import Transformer2DModel
import matplotlib.pyplot as plt

# Add Google Drive to Python path
sys.path.append('/content/drive/MyDrive/Mirrors')
import mirrors_helper_functions


# TODO:Just for development, remove in the end
import importlib
importlib.reload(mirrors_helper_functions)

<module 'mirrors_helper_functions' from '/content/drive/MyDrive/Mirrors/mirrors_helper_functions.py'>

In [ ]:
# TODO: Use the refiner from here: https://huggingface.co/stabilityai/stable-diffusion-xl-base-1.0

# Load the pipeline
pipe = DiffusionPipeline.from_pretrained("stabilityai/stable-diffusion-xl-base-1.0", torch_dtype=torch.float16, use_safetensors=True, variant="fp16")
# pipe.__call__ = mirrors_helper_functions.new_pipe_call.__get__(pipe, StableDiffusionXLPipeline)
# pipe.to("cuda")

i = 0
# Attach the hook to all cross-attention layers
for name, module in pipe.unet.named_modules():
    if name.endswith("attn2") and isinstance(module, Attention):
      module.forward = mirrors_helper_functions.new_forward.__get__(module, Transformer2DModel)
      module.processor.concatenated_attention_maps = None
      i += 1

pipe.encode_prompt = mirrors_helper_functions.new_encode_prompt.__get__(pipe, StableDiffusionXLPipeline)

# Define the prompt and generate an image
prompt = "A man in front of a mirror"
image = pipe(prompt=prompt).images[0]

# Visualize the generated image
plt.imshow(image)
plt.axis('off')
plt.show()



/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

tokenized text_inputs is: {'input_ids': tensor([[49406,   320,   786,   530,  2184,   539,   320,  7220, 49407, 49407,
         49407, 49407, 49407, 49407, 49407, 49407, 49407, 49407, 49407, 49407,
         49407, 49407, 49407, 49407, 49407, 49407, 49407, 49407, 49407, 49407,
         49407, 49407, 49407, 49407, 49407, 49407, 49407, 49407, 49407, 49407,
         49407, 49407, 49407, 49407, 49407, 49407, 49407, 49407, 49407, 49407,
         49407, 49407, 49407, 49407, 49407, 49407, 49407, 49407, 49407, 49407,
         49407, 49407, 49407, 49407, 49407, 49407, 49407, 49407, 49407, 49407,
         49407, 49407, 49407, 49407, 49407, 49407, 49407]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0]])}
text_input_ids is: tensor([[49406,   320,   786,   530,  2

  0%|          | 0/50 [00:00<?, ?it/s]

encoder_hidden_states shape is: torch.Size([2, 77, 2048]) and its:
tensor([[[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         ...,
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]],

        [[-3.8926, -2.5117,  4.7109,  ...,  0.1899,  0.4185, -0.2971],
         [ 0.0890, -0.6182, -0.4883,  ...,  0.5010, -0.0375, -0.1567],
         [ 0.4390,  0.5884, -0.5259,  ..., -0.2102,  0.4382, -0.2722],
         ...,
         [ 0.2549,  0.0526,  0.2710,  ...,  0.1750,  0.2734,  0.5273],
         [ 0.2583,  0.0439,  0.2729,  ...,  0.1509,  0.1650,  0.4675],
         [ 0.2418,  0.0792,  0.2590,  ..., -0.0643,  0.1931,  0.4736]]],
       dtype=torch.float16)
encoder_hidden_states

In [ ]:
import inspect
# print(pipe.text_encoder)
# print(pipe.tokenizer)
text_inputs = pipe.tokenizer(
                    prompt,
                    padding="max_length",
                    max_length=pipe.tokenizer.model_max_length,
                    truncation=True,
                    return_tensors="pt",
                )
print(text_inputs)
print(f'text_inputs["input_ids"].shape is: {text_inputs["input_ids"].shape}')
print(f'text_inputs["attention_mask"].shape is: {text_inputs["attention_mask"].shape}\n')
prompt_embeds_list = []
prompt_embeds = pipe.text_encoder(text_inputs["input_ids"].to(pipe._execution_device), output_hidden_states=True)
prompt_embeds = prompt_embeds.hidden_states[-2]
print(f'prompt_embeds after encoding shape is: {prompt_embeds.shape} and its actually\n{prompt_embeds}\n')
prompt_embeds_list.append(prompt_embeds)
prompt_embeds = torch.concat(prompt_embeds_list, dim=-1)
prompt_embeds = prompt_embeds.to(dtype=pipe.unet.dtype, device=pipe._execution_device)
bs_embed, seq_len, _ = prompt_embeds.shape
prompt_embeds = prompt_embeds.repeat(1, 1, 1)
prompt_embeds = prompt_embeds.view(bs_embed * 1, seq_len, -1)
print(f'prompt_embeds after all (This is what injected to the unet as "encoder_hidden_states") shape is: {prompt_embeds.shape} and its actually\n{prompt_embeds}\n')

In [ ]:
print(inspect.getsource(pipe.prepare_latents))

In [ ]:
print(inspect.getsource(pipe.unet.__call__))
print(pipe.unet)

In [ ]:
for downsample_block in pipe.unet.down_blocks:
  print(inspect.getsource(downsample_block.forward) + "/n/n/n/n")